In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.path.dirname(os.getcwd()))

from Value import Value
from utils import draw_trace

Veamos el siguiente bug que ocurre en la implementación actual de nuestro backpropagation:

In [ ]:
a = Value(3.0, label='a')
b = a + a; b.label = 'b'
b.backward()
draw_trace(b)

El grafo no muestra todo, en realidad, hay dos nodos con el label 'a' solapados. Lo que sí está mal
es que el gradiente de `b` con respecto a `a` sea 1. Si lo calculamos analíticamente, vemos que
debería ser 2:

Si $b = f(a) = a + a$, entonces $\partial{b}/\partial{a} = f'(a) = 1 + 1 = 2$

Veamos por qué ocurre esto:
`b` es el resultado de una suma, entonces al llamar a `b._backward()` (esto ocurre al llamar a `b.backward()`), se está llamando a la siguiente función:
```python
def _backward():
    self.grad = 1.0 * out.grad
    other.grad = 1.0 * out.grad
```

En este caso, `out.grad` es 1 (esto se sete al llamar `b.backward()`). Y lo que pasa es que `self` y `other` son ambos el mismo `Value` (que es `a`), entonces
estamos sobreescribiendo el gradiente.

Veámoslo con una expresión más complicada:

In [ ]:
a = Value(-2.0, label='a')
b = Value(3.0, label='b')
d = a * b; d.label = 'd'
e = a + b; e.label = 'e'
f = d * e; f.label = 'f'

f.backward()

draw_trace(f)

Calculemos los gradientes de `a` y `b` analíticamente:

Tenemos que:
$$
\begin{align*}
    f &= d * e \\
      &= (a * b) * (a + b)
\end{align*}
$$

Entonces, tenemos que aplicar la regla de la cadena viendo a f como $ f(d(a,b), (e(a,b)))$. Estamos en este caso: [Chain rule - Case of scalar-valued functions with multiple inputs](https://en.wikipedia.org/wiki/Chain_rule#Case_of_scalar-valued_functions_with_multiple_inputs).

$$
\begin{align*}
    \frac{\partial{f}}{\partial{a}} &= \frac{\partial{f}}{\partial{d}}\frac{\partial{d}}{\partial{a}} + \frac{\partial{f}}{\partial{e}}\frac{\partial{e}}{\partial{a}} \\
    &= e * b + d * 1 \\
    &= e * b + d \\
    &= 1 * 3 + (-6) \\
    &= -3
\end{align*}
$$

Los gradientes de `e` y `d` están bien. Los que están mal son los de `a` y `b`, esta vez porque se está usando la misma referencia en dos operaciones distintas. Entonces, supongamos que primero se calcula el gradiente de `a` y `b` pasando por `e`, pero después al hacer lo mismo pasando por `d`, los valores anteriores se sobreescriben.

Entonces, básicamente lo que va a pasar es que siempre que usemos un mismo `Value` más de una vez, vamos a tener un error.

La solución a esto es ir acumulando el gradiente, como explica en [Chain rule - Multivariable case](https://en.wikipedia.org/wiki/Chain_rule#Multivariable_case).

Lo aplicamos en la clase `Value` y si corremos este código de nuevo, vamos a ver que ahora los gradientes dan bien.